In [ ]:
!pip install gdown

In [ ]:
import gdown
gdown.download('https://drive.google.com/uc?id=1yThAC-VoMRVDUu7GWCGO8uVhxWF6w5q2', 'train.csv', quiet=False)

In [ ]:
import pandas as pd
train_path = 'train.csv'
df = pd.read_csv(train_path)

# df 

In [2]:
# 모든 컬럼명 출력
# print(df.columns.tolist())
print(len(df.columns))


122


In [7]:
from scipy.stats import anderson, shapiro
import warnings
import numpy as np

# FutureWarning 방지 설정
warnings.simplefilter(action='ignore', category=FutureWarning)

# 수치형 변수와 범주형 변수 구분
numerical_cols = df.select_dtypes(include=[np.number]).columns
categorical_cols = df.select_dtypes(include=['object']).columns

# 수치형 변수 결측치 처리
for col in numerical_cols:
    # 결측치가 있는 경우에만 진행
    if df[col].isnull().sum() > 0:
        # 샘플 수가 5000을 초과하면 Anderson-Darling 테스트 사용, 그렇지 않으면 Shapiro-Wilk 테스트 사용
        if len(df[col].dropna()) > 5000:
            result = anderson(df[col].dropna(), dist='norm')
            if result.statistic < result.critical_values[2]:
                mean_value = df[col].mean()
                df[col] = df[col].fillna(mean_value).copy()
                print(f"Column '{col}'의 결측치를 평균값으로 대체하였습니다 (Anderson-Darling 테스트 사용).")
            else:
                median_value = df[col].median()
                df[col] = df[col].fillna(median_value).copy()
                print(f"Column '{col}'의 결측치를 중앙값으로 대체하였습니다 (Anderson-Darling 테스트 사용).")
        else:
            # 정규성 테스트 (Shapiro-Wilk Test 사용)
            stat, p = shapiro(df[col].dropna())
            # p-value가 0.05보다 크다면 데이터가 정규분포를 따를 가능성이 높다고 판단하여 평균값으로 대체
            if p > 0.05:
                mean_value = df[col].mean()
                df[col] = df[col].fillna(mean_value).copy()
                print(f"Column '{col}'의 결측치를 평균값으로 대체하였습니다.")
            else:
                # 정규분포가 아니거나 이상치가 많은 경우 중앙값으로 대체
                median_value = df[col].median()
                df[col] = df[col].fillna(median_value).copy()
                print(f"Column '{col}'의 결측치를 중앙값으로 대체하였습니다.")

# 범주형 변수 결측치 처리
for col in categorical_cols:
    # 결측치가 있는 경우 최빈값으로 대체
    if df[col].isnull().sum() > 0:
        mode_value = df[col].mode()[0]
        df[col] = df[col].fillna(mode_value).copy()
        print(f"Column '{col}'의 결측치를 최빈값으로 대체하였습니다.")

Column 'AMT_ANNUITY'의 결측치를 중앙값으로 대체하였습니다 (Anderson-Darling 테스트 사용).
Column 'AMT_GOODS_PRICE'의 결측치를 중앙값으로 대체하였습니다 (Anderson-Darling 테스트 사용).
Column 'OWN_CAR_AGE'의 결측치를 중앙값으로 대체하였습니다 (Anderson-Darling 테스트 사용).
Column 'CNT_FAM_MEMBERS'의 결측치를 중앙값으로 대체하였습니다 (Anderson-Darling 테스트 사용).
Column 'EXT_SOURCE_1'의 결측치를 중앙값으로 대체하였습니다 (Anderson-Darling 테스트 사용).
Column 'EXT_SOURCE_2'의 결측치를 중앙값으로 대체하였습니다 (Anderson-Darling 테스트 사용).
Column 'EXT_SOURCE_3'의 결측치를 중앙값으로 대체하였습니다 (Anderson-Darling 테스트 사용).
Column 'APARTMENTS_AVG'의 결측치를 중앙값으로 대체하였습니다 (Anderson-Darling 테스트 사용).
Column 'BASEMENTAREA_AVG'의 결측치를 중앙값으로 대체하였습니다 (Anderson-Darling 테스트 사용).
Column 'YEARS_BEGINEXPLUATATION_AVG'의 결측치를 중앙값으로 대체하였습니다 (Anderson-Darling 테스트 사용).
Column 'YEARS_BUILD_AVG'의 결측치를 중앙값으로 대체하였습니다 (Anderson-Darling 테스트 사용).
Column 'COMMONAREA_AVG'의 결측치를 중앙값으로 대체하였습니다 (Anderson-Darling 테스트 사용).
Column 'ELEVATORS_AVG'의 결측치를 중앙값으로 대체하였습니다 (Anderson-Darling 테스트 사용).
Column 'ENTRANCES_AVG'의 결측치를 중앙값으로 대체하였습니다 (Anderson-Darling 테스트 사용).
Co

In [9]:
# 결측치가 남아 있는지 확인
print("Remaining missing values:\n", df.isnull().sum().sort_values(ascending=False))


Remaining missing values:
 SK_ID_CURR                     0
OBS_30_CNT_SOCIAL_CIRCLE       0
WALLSMATERIAL_MODE             0
TOTALAREA_MODE                 0
HOUSETYPE_MODE                 0
                              ..
LIVE_REGION_NOT_WORK_REGION    0
REG_REGION_NOT_WORK_REGION     0
REG_REGION_NOT_LIVE_REGION     0
HOUR_APPR_PROCESS_START        0
AMT_REQ_CREDIT_BUREAU_YEAR     0
Length: 122, dtype: int64


In [10]:
def find_outliers_and_replace(df, whis=1.5):
    """
    주어진 데이터프레임에서 숫자형 열들에 대해 IQR 방식을 통해 이상치를 찾아
    이상치 값을 최소 또는 최대 허용 범위 값으로 대체하는 함수
    
    Parameters:
    - df: 데이터프레임
    - whis: IQR의 배수로, 기본값은 1.5
    
    Returns:
    - 이상치가 대체된 데이터프레임
    """
    # 숫자형 열만 선택
    numeric_columns = df.select_dtypes(include='number').columns
    
    for column_name in numeric_columns:
        # 1사분위수(Q1)와 3사분위수(Q3) 계산
        q1, q3 = df[column_name].quantile([0.25, 0.75])
        iqr = q3 - q1

        # 하한과 상한 계산
        lower_bound = q1 - iqr * whis
        upper_bound = q3 + iqr * whis

        # 이상치를 하한과 상한으로 대체
        df[column_name] = df[column_name].apply(lambda x: lower_bound if x < lower_bound else (upper_bound if x > upper_bound else x))
    
    return df

In [11]:
df = find_outliers_and_replace(df)
df.head(10) 

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,0.0,Cash loans,M,N,Y,0.0,202500.0,406597.5,24700.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0.0,Cash loans,F,N,N,0.0,270000.0,1293502.5,35698.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0.0,Revolving loans,M,Y,Y,0.0,67500.0,135000.0,6750.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0.0,Cash loans,F,N,Y,0.0,135000.0,312682.5,29686.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,100007,0.0,Cash loans,M,N,Y,0.0,121500.0,513000.0,21865.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,100008,0.0,Cash loans,M,N,Y,0.0,99000.0,490495.5,27517.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6,100009,0.0,Cash loans,F,Y,Y,1.0,171000.0,1560726.0,41301.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
7,100010,0.0,Cash loans,M,Y,Y,0.0,337500.0,1530000.0,42075.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,100011,0.0,Cash loans,F,N,Y,0.0,112500.0,1019610.0,33826.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9,100012,0.0,Revolving loans,M,N,Y,0.0,135000.0,405000.0,20250.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [12]:
df.shape

(307511, 122)

In [15]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# 원본 데이터프레임 컬럼 수
original_column_count = df.shape[1]

# 범주형 및 True/False 컬럼 선택
categorical_columns = df.select_dtypes(include=['object', 'category']).columns
bool_columns = df.select_dtypes(include='bool').columns

# 클래스가 2개인 컬럼에는 Label Encoding, 3개 이상인 컬럼에는 One-Hot Encoding
for col in categorical_columns:
    unique_classes = df[col].nunique()  # 해당 컬럼의 클래스 개수 확인
    if unique_classes == 2:
        # Label Encoding 적용
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
    elif unique_classes > 2:
        # One-Hot Encoding 적용 후 생성된 컬럼에 대해 Label Encoding 적용
        df = pd.get_dummies(df, columns=[col], prefix=col)

# bool 타입 컬럼을 0과 1로 변환
for col in bool_columns:
    df[col] = df[col].astype(int)

# One-Hot Encoding 후 새로 생긴 컬럼 선택 (prefix 기반으로 선택할 수 있음)
onehot_encoded_columns = [col for col in df.columns if '_' in col]

# One-Hot Encoding 후 생성된 컬럼들에 Label Encoding 적용
for col in onehot_encoded_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

# 인코딩 후 증가된 컬럼 수 계산
new_column_count = df.shape[1]
added_columns = new_column_count - original_column_count

# 결과 출력
print(f"Number of added columns after encoding: {added_columns}")
print("Top 10 rows of the encoded dataframe:")
df.head(10)


Number of added columns after encoding: 0
Top 10 rows of the encoded dataframe:


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,HOUSETYPE_MODE_block of flats,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden
0,0,0.0,0,0,1,0,1754,1191,4588,189,...,1,0,0,0,0,0,0,0,1,0
1,1,0.0,0,0,0,0,2064,4338,6985,704,...,1,0,0,1,0,0,0,0,0,0
2,2,0.0,1,1,1,0,338,228,662,30,...,1,0,0,0,0,0,0,1,0,0
3,3,0.0,0,0,1,0,1170,836,5680,145,...,1,0,0,0,0,0,0,1,0,0
4,4,0.0,0,0,1,0,1019,1635,3960,352,...,1,0,0,0,0,0,0,1,0,0
5,5,0.0,0,0,1,0,719,1533,5208,294,...,1,0,0,0,0,0,0,1,0,0
6,6,0.0,0,1,1,1,1519,4882,8167,780,...,1,0,0,0,0,0,0,1,0,0
7,7,0.0,0,1,1,0,2209,4836,8324,780,...,1,0,0,0,0,0,0,1,0,0
8,8,0.0,0,0,1,0,904,3471,6582,621,...,1,0,0,0,0,0,0,1,0,0
9,9,0.0,1,0,1,0,1170,1185,3603,241,...,1,0,0,0,0,0,0,1,0,0


In [16]:
df.shape

(307511, 242)

In [18]:
df.to_csv('trainset.csv', index=False)